In [1]:
%pip install numpy opencv-python mediapipe

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.2/38.8 MB 5.3 MB/s eta 0:00:08
   ---------------------------------------- 0.3/38.8 MB 4.2 MB/s eta 0:00:10
    --------------------------------------- 0.5/38.8 MB 3.9 MB/s eta 0:00:10
    --------------------------------------- 0.7/38.8 MB 3.8 MB/s eta 0:00:11
    --------------------------------------- 0.9/38.8 MB 3.6 MB/s eta 0:00:11
   - -------------------------------------- 1.0/38.8 MB 3.6 MB/s eta 0:00:11
   - -------------------------------------- 1.2/38.8 MB 3.5 MB/s eta 0:00:11
   - -------------------------------------- 1.3/38.8 MB 3.5 MB/s eta 0:00:11
   - -------------------------------------- 1.5/38.8 MB 3.5 MB/s eta 0:00:11
   - -------------------------------------- 1.6/38.8 MB 3.5 MB/s eta 0:00:11
   - -------------------------------------- 1.8/38.8 MB 3.5 MB/s eta 0:00:11
   

In [2]:
import numpy as np
import mediapipe as mp
import cv2

In [3]:
mph=mp.solutions.holistic
mpd=mp.solutions.drawing_utils

In [4]:
def dl(image,results):
    mpd.draw_landmarks(image,results.face_landmarks,mph.FACEMESH_TESSELATION,mpd.DrawingSpec(color=(34,234,5),thickness=1,circle_radius=1), mpd.DrawingSpec(color=(34,123,90),thickness=1,circle_radius=1))
    mpd.draw_landmarks(image,results.pose_landmarks,mph.POSE_CONNECTIONS, mpd.DrawingSpec(color=(23,5,100),thickness=2,circle_radius=3), mpd.DrawingSpec(color=(123,80,67),thickness=2,circle_radius=2))
    mpd.draw_landmarks(image,results.left_hand_landmarks,mph.HAND_CONNECTIONS,mpd.DrawingSpec(color=(90,78,56),thickness=2,circle_radius=4), mpd.DrawingSpec(color=(34,123,90),thickness=2,circle_radius=2))
    mpd.draw_landmarks(image,results.right_hand_landmarks,mph.HAND_CONNECTIONS, mpd.DrawingSpec(color=(67,34,90),thickness=2,circle_radius=4), mpd.DrawingSpec(color=(23,4,56),thickness=2,circle_radius=2))


In [5]:
def detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

In [6]:
mpd.draw_landmarks??

Signature:
mpd.draw_landmarks(
    image: numpy.ndarray,
    landmark_list: mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList,
    connections: Optional[List[Tuple[int, int]]] = None,
    landmark_drawing_spec: Union[mediapipe.python.solutions.drawing_utils.DrawingSpec, Mapping[int, mediapipe.python.solutions.drawing_utils.DrawingSpec], NoneType] = DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
    connection_drawing_spec: Union[mediapipe.python.solutions.drawing_utils.DrawingSpec, Mapping[Tuple[int, int], mediapipe.python.solutions.drawing_utils.DrawingSpec]] = DrawingSpec(color=(224, 224, 224), thickness=2, circle_radius=2),
    is_drawing_landmarks: bool = True,
)
Source:   
def draw_landmarks(
    image: np.ndarray,
    landmark_list: landmark_pb2.NormalizedLandmarkList,
    connections: Optional[List[Tuple[int, int]]] = None,
    landmark_drawing_spec: Optional[
        Union[DrawingSpec, Mapping[int, DrawingSpec]]
    ] = DrawingSpec(color=RED_COLOR),

In [7]:
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [88]:
#from matplotlib import pyplot

In [89]:
#pyplot.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [8]:
def arr(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([pose,face,left_hand,right_hand])

In [9]:

import os 

In [20]:
words = [
    "Hello",
    "I",
    "Yes",
    "No",
    "Please",
    "Thank you",
    "Want",
    "What",
    "Home",
    "Eat food",
    "More",
    "Bathroom",
    "Fine",
    "Like",
    "Finish",
    
]

In [21]:
len(words)

15

In [29]:
PATH_DIRECTORY = os.path.join('Dataset')
signs = np.array(words) 
total_number_of_sequences = 40
frame_length = 50
'''
for sign in signs:
    for sequence in range(total_number_of_sequences):
        try:
            os.makedirs(os.path.join(PATH_DIRECTORY,sign,str(sequence)))
        except:
            pass
'''

'\nfor sign in signs:\n    for sequence in range(total_number_of_sequences):\n        try:\n            os.makedirs(os.path.join(PATH_DIRECTORY,sign,str(sequence)))\n        except:\n            pass\n'

In [22]:
import json

In [23]:
BASE_DIRECTORY = 'Dataset'
PROGRESS_FILE = os.path.join(BASE_DIRECTORY, 'progress.json')

In [16]:
def load_progress():
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as f:
            return json.load(f)
    return {'sign': None, 'sequence': 0, 'frame': 0,'sign_index':0}

In [17]:
def save_progress(progress):
    with open(PROGRESS_FILE, 'w') as f:
        json.dump(progress, f)

In [98]:
'''capture = cv2.VideoCapture(0)
with mph.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as model:
    for sign in signs:
        for sequence in range(total_number_of_sequences):
            for fnum in range(frame_length):
                ret, frame = capture.read()
                image,results=detection(frame,model)
                dl(image,results)

                if fnum == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(sign, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('Sign Language Recognition', image)
                    cv2.waitKey(5000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(sign, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('Sign Language Recognition', image)


                keypoints = arr(results)
                npy_file = os.path.join(PATH_DIRECTORY, sign, str(sequence), str(fnum))
                np.save(npy_file, keypoints)

                #cv2.imshow('Sign Language Recognition',image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    capture.release()
    cv2.destroyAllWindows()
''' 
#without collection progress

"capture = cv2.VideoCapture(0)\nwith mph.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as model:\n    for sign in signs:\n        for sequence in range(total_number_of_sequences):\n            for fnum in range(frame_length):\n                ret, frame = capture.read()\n                image,results=detection(frame,model)\n                dl(image,results)\n\n                if fnum == 0:\n                    cv2.putText(image, 'STARTING COLLECTION', (120,200), \n                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)\n                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(sign, sequence), (15,12), \n                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)\n                    cv2.imshow('Sign Language Recognition', image)\n                    cv2.waitKey(5000)\n                else:\n                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.

In [18]:
capture = cv2.VideoCapture(0)

with mph.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as model:

    progress = load_progress()
    start_sign = progress.get('sign', None)
    start_sequence = progress.get('sequence', 0)
    start_frame = progress.get('frame', 0)
    start_index = progress.get('sign_index', 0)
    

    for i, sign in enumerate(signs[start_index:], start=start_index):
        if sign == start_sign:
            sequence_start = start_sequence
        else:
            sequence_start = 0

        for sequence in range(sequence_start, total_number_of_sequences):
            if sign == start_sign and sequence == start_sequence:
                frame_start = start_frame
            else:
                frame_start = 0

            for fnum in range(frame_start, frame_length):
                ret, frame = capture.read()
                if not ret:
                    break
                
                image, results = detection(frame, model)
                dl(image, results)

                if fnum == 0:
                    cv2.putText(image, 'DATASET COLLECTING', (120, 200),
                                cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting dataset {} Video Number {} frame number {}'.format(sign, sequence, fnum), (15, 12),
                                cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('Sign Language Recognition', image)
                    cv2.waitKey(1000)
                else:
                    cv2.putText(image, 'Collecting dataset {} Video Number {} frame number {}'.format(sign, sequence, fnum), (15, 12),
                                cv2.FONT_HERSHEY_TRIPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('Sign Language Recognition', image)

                keypoints = arr(results)
                npy_file = os.path.join(PATH_DIRECTORY, sign, str(sequence), str(fnum))
                np.save(npy_file, keypoints)

                # Save progress after each frame
                save_progress({'sign': sign, 'sequence': sequence, 'frame': fnum, 'sign_index': i})

                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    print(f"Quitting... Progress saved. Last processed sign: '{sign}', sequence: {sequence}, frame: {fnum}")
                    break
            if key == ord('q'):
                break
        if key == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()


Quitting... Progress saved. Last processed sign: 'Hello', sequence: 1, frame: 24


In [19]:
#for force shutdown
#capture.release()
#cv2.destroyAllWindows()

In [25]:
%pip install tensorflow

     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     --------------------------------- ------ 41.0/48.7 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 48.7/48.7 kB 409.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/385.2 MB 2.6 MB/s eta 0:02:26
   ---------------------------------------- 0.2/385.2 MB 2.8 MB/s eta 0:02:20
   ---------------------------------------- 0.3/385.2 MB 2.6 MB/s eta 0:02:27
   ---------------------------------------- 0.4/385.2 MB 2.5 MB/s eta 0:02:33
   ---------------------------------------- 0.5/385.2 MB 2.0 MB/s eta 0:03:11
   ---------------------------------------- 0.5/385.2 MB 2.0 MB/s eta 0:03:16
   ---------------------------------------- 0.6/385.2 MB 1.9 MB/s eta 0:03:19
   ---------------------------------------- 0.7/385.2 MB 2.0 MB/s eta 0:03:13
   ---------------------------------------- 0.8/385.2 MB 2.0 MB/s eta 0:0

In [26]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [31]:
label_map = {label:num for num, label in enumerate(signs)}

In [32]:
label_map

{'Hello': 0,
 'I': 1,
 'Yes': 2,
 'No': 3,
 'Please': 4,
 'Thank you': 5,
 'Want': 6,
 'What': 7,
 'Home': 8,
 'Eat food': 9,
 'More': 10,
 'Bathroom': 11,
 'Fine': 12,
 'Like': 13,
 'Finish': 14}

In [33]:
sequences, labels = [], []
for action in signs:
    for sequence in np.array(os.listdir(os.path.join(PATH_DIRECTORY, action))).astype(int):
        window = []
        for frame_num in range(frame_length):
            res = np.load(os.path.join(PATH_DIRECTORY, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [34]:
np.array(sequences).shape

(600, 50, 1662)

In [35]:
np.array(labels).shape

(600,)

In [36]:
X = np.array(sequences)

In [37]:
X.shape

(600, 50, 1662)

In [38]:
y = to_categorical(labels).astype(int)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [40]:
y_test.shape

(30, 15)

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [48]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [49]:
# Define the model
model = Sequential()

# First LSTM layer
model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(Dropout(0.3))  # Add dropout for regularization
model.add(BatchNormalization())

# Second LSTM layer
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

# Third LSTM layer
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

# Dense layers
model.add(Dense(64, activation='relu', kernel_regularizer='l2'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu', kernel_regularizer='l2'))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(signs.shape[0], activation='softmax'))

In [50]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [51]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [52]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 117ms/step - accuracy: 0.0656 - loss: 4.1246
Epoch 2/2000
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 123ms/step - accuracy: 0.0686 - loss: 3.9161
Epoch 3/2000
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 130ms/step - accuracy: 0.0634 - loss: 3.8214
Epoch 4/2000
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 133ms/step - accuracy: 0.0655 - loss: 3.7430
Epoch 5/2000
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step - accuracy: 0.0464 - loss: 3.6923
Epoch 6/2000
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 112ms/step - accuracy: 0.0717 - loss: 3.6027
Epoch 7/2000
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 115ms/step - accuracy: 0.0520 - loss: 3.5341
Epoch 8/2000
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 116ms/step - accuracy: 0.0765 - loss: 3.5046
Epoch 9/2000
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 117ms/step - accuracy: 0.0849 - loss: 3.4733
Epoch 10/2000
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - accuracy: 0.0632 - loss: 3.4261
Epoch 11/2000
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - accuracy: 0.0617 - loss: 3.4417
Epoch 12/2000
18/18 ━━━━━━━━━

KeyboardInterrupt: 